In [1]:
from pathlib import Path

from utilities import (
    extract_audio_duration,
    display_verses_with_timing,
    create_ass_file,
    extract_lyrics_with_timing,
    generate_karaoke_video
)

c:\Users\chris\anaconda3\envs\audio_app\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vocal_file_path = Path("./audio_processing/karaoke_files/input_vocals/")
vocals = vocal_file_path / "droppin_seeds.mp3"

verses = extract_lyrics_with_timing(str(vocals))

verses

[{'verse_number': 1,
  'text': ' like  adam  in  the  garden  of  eve  my  bitch  got  an  apple  bottom  and  she  swallowed  my  seeds  follow',
  'start': 11.959999999999996,
  'end': 17.86,
  'words': [{'word': ' like',
    'word_number': 1,
    'start': 11.959999999999996,
    'end': 12.539999999999997,
    'probability': 0.052825927734375},
   {'word': ' adam',
    'word_number': 2,
    'start': 12.539999999999997,
    'end': 13.12,
    'probability': 0.467529296875},
   {'word': ' in',
    'word_number': 3,
    'start': 13.12,
    'end': 13.34,
    'probability': 0.92236328125},
   {'word': ' the',
    'word_number': 4,
    'start': 13.34,
    'end': 13.44,
    'probability': 0.97412109375},
   {'word': ' garden',
    'word_number': 5,
    'start': 13.44,
    'end': 13.84,
    'probability': 0.9716796875},
   {'word': ' of',
    'word_number': 6,
    'start': 13.84,
    'end': 14.02,
    'probability': 0.908203125},
   {'word': ' eve',
    'word_number': 7,
    'start': 14.02,
 

In [3]:
from pprint import pprint

pprint(verses[0])

{'end': 17.86,
 'start': 11.959999999999996,
 'text': ' like  adam  in  the  garden  of  eve  my  bitch  got  an  apple  '
         'bottom  and  she  swallowed  my  seeds  follow',
 'verse_number': 1,
 'words': [{'end': 12.539999999999997,
            'probability': 0.052825927734375,
            'start': 11.959999999999996,
            'word': ' like',
            'word_number': 1},
           {'end': 13.12,
            'probability': 0.467529296875,
            'start': 12.539999999999997,
            'word': ' adam',
            'word_number': 2},
           {'end': 13.34,
            'probability': 0.92236328125,
            'start': 13.12,
            'word': ' in',
            'word_number': 3},
           {'end': 13.44,
            'probability': 0.97412109375,
            'start': 13.34,
            'word': ' the',
            'word_number': 4},
           {'end': 13.84,
            'probability': 0.9716796875,
            'start': 13.44,
            'word': ' garden',
       

In [ ]:
display_verses_with_timing(verses[:2])

___
___
___

In [4]:
# Paths to areas being used

# font_file_path = Path("./audio_processing/karaoke_files/input_fonts/")
# lyrics_file_path = Path("./audio_processing/karaoke_files/input_lyrics/")

instrumental_file_path = Path("./audio_processing/karaoke_files/input_instrumentals/")
output_video_path = Path("./audio_processing/karaoke_files/output_videos/")
output_ass_path = Path("./audio_processing/karaoke_files/output_ass/")

# Actual file in those paths

# karaoke_font = font_file_path / "enter-the-gungeon-big.ttf"
karaoke_music = instrumental_file_path / "droppin_seeds.mp3"
karaoke_video = output_video_path / "droppin_seeds_karaoke.mp4"

# Verses metadata.
karaoke_lyrics = verses

In [5]:
audio_duration = extract_audio_duration(karaoke_music)

audio_duration

60.107755

In [6]:
create_ass_file(
    verses,
    output_path= output_ass_path / "droppin_seeds_karaoke.ass",
    audio_duration=audio_duration,
    font="Arial",
    fontsize=10,
    title="test karaoke",
    # loader_duration=True,
)

___
___
___

In [7]:
ass_path = output_ass_path / "droppin_seeds_karaoke.ass"

generate_karaoke_video(
    audio_path=str(karaoke_music),
    ass_path=str(ass_path).replace("\\", "/"),
    output_path=str(karaoke_video),
)


✅ GPU detected: NVIDIA GeForce RTX 3060 Ti

Running FFmpeg command:
ffmpeg -y -f lavfi -i color=c=black:s=1280x720:d=60.107755 -i audio_processing\karaoke_files\input_instrumentals\droppin_seeds.mp3 -vf subtitles=audio_processing/karaoke_files/output_ass/droppin_seeds_karaoke.ass -c:v h264_nvenc -preset fast -crf 23 -r 24 -b:v 3000k -c:a aac -b:a 192k -shortest audio_processing\karaoke_files\output_videos\droppin_seeds_karaoke.mp4
✅ Video successfully created at: audio_processing\karaoke_files\output_videos\droppin_seeds_karaoke.mp4
